In [1]:
using PyPlot, TimeIt 
import PyPlot
const plt = PyPlot
using Distributions, Optim
include("ZZ_clustering.jl") 

ZZ_clustering (generic function with 5 methods)

In [2]:
function logistic_data(d, Nobs, p=0.1, δ=3, m=10) 
    X = rand(d, Nobs)
    X[:,1:m] = δ*rand(Normal(), d, m)
    y = rand(Binomial(1,p), Nobs)
    return X, y
end

logistic_data (generic function with 4 methods)

In [3]:
d, Nobs = 10, 100
σ = 2
p = 0.5
δ = 1
m = 50
X, y = logistic_data(d, Nobs, p, δ, m)
root = find_root(X, y, σ);

In [4]:
β_0 = copy(root)
mb_size = 10
A = eye(d)
replace = true
max_attempts = 10^6;

In [5]:
start = time()
s_kmeans, b_kmeans = ZZ_clustering(X, y, max_attempts, β_0, mb_size, root, σ, A, "kmeans", false) 
s_sorted_uw, b_sorted_uw = ZZ_clustering(X, y, max_attempts, β_0, mb_size, root, σ, A, "sorted", false) 
s_sorted_w, b_sorted_w = ZZ_clustering(X, y, max_attempts, β_0, mb_size, root, σ, A, "sorted", true) 
print( round((time()-start)/60, 2), " minutes to run")

1.7% of switches accepted 
0.68% of switches accepted 
1.9% of switches accepted 
1.04 minutes to run

In [6]:
Nsamples = 10^4
samples_kmeans = extract_samples(s_kmeans, b_kmeans, b_kmeans[end]/Nsamples)
samples_sorted_uw = extract_samples(s_sorted_uw, b_sorted_uw, b_sorted_uw[end]/Nsamples)
samples_sorted_w = extract_samples(s_sorted_w, b_sorted_w, b_sorted_w[end]/Nsamples)
print(round.([compute_configT(samples_kmeans, i, X, y, Nobs, σ) for i in 1:d], 2), "\n")
print(round.([compute_configT(samples_sorted_w, i, X, y, Nobs, σ) for i in 1:d], 2), "\n")
print(round.([compute_configT(samples_sorted_uw, i, X, y, Nobs, σ) for i in 1:d], 2))

[1.03, 0.99, 1.0, 0.99, 1.07, 0.93, 1.0, 0.91, 1.1, 0.98]
[0.99, 0.94, 1.14, 1.0, 0.95, 1.04, 1.06, 0.96, 1.0, 0.99]
[0.91, 0.96, 1.02, 0.96, 1.15, 0.86, 1.1, 0.97, 0.99, 0.89]